# Module 4 Graded Quiz

In [1]:
import pandas as pd
import numpy as np

In [2]:
def discount(t, r):
    """
    Compute the price of a pure discount bond that pays a dollar at time period t
    and r is the per-period interest rate
    returns a |t| x |r| Series or DataFrame
    r can be a float, Series or DataFrame
    returns a DataFrame indexed by t
    """
    discounts = pd.DataFrame([(r+1)**-i for i in t])
    discounts.index = t
    return discounts

def pv(flows, r):
    """
    Compute the present value of a sequence of cash flows given by the time (as an index) and amounts
    r can be a scalar, or a Series or DataFrame with the number of rows matching the num of rows in flows
    """
    dates = flows.index
    discounts = discount(dates, r)
    return discounts.multiply(flows, axis='rows').sum()

def bond_cash_flows(maturity, principal=100, coupon_rate=0.03, coupons_per_year=12):
    """
    Returns the series of cash flows generated by a bond,
    indexed by the payment/coupon number
    """
    n_coupons = round(maturity*coupons_per_year)
    coupon_amt = principal*coupon_rate/coupons_per_year
    coupons = np.repeat(coupon_amt, n_coupons)
    coupon_times = np.arange(1, n_coupons+1)
    cash_flows = pd.Series(data=coupon_amt, index=coupon_times)
    cash_flows.iloc[-1] += principal
    return cash_flows

def bond_price(maturity, principal=100, coupon_rate=0.03, coupons_per_year=12, discount_rate=0.03):
    """
    Computes the price of a bond that pays regular coupons until maturity
    at which time the principal and the final coupon is returned
    This is not designed to be efficient, rather,
    it is to illustrate the underlying principle behind bond pricing!
    If discount_rate is a DataFrame, then this is assumed to be the rate on each coupon date
    and the bond value is computed over time.
    i.e. The index of the discount_rate DataFrame is assumed to be the coupon number
    """
    if isinstance(discount_rate, pd.DataFrame):
        pricing_dates = discount_rate.index
        prices = pd.DataFrame(index=pricing_dates, columns=discount_rate.columns)
        for t in pricing_dates:
            prices.loc[t] = bond_price(maturity-t/coupons_per_year, principal, coupon_rate, coupons_per_year,
                                      discount_rate.loc[t])
        return prices
    else: # base case ... single time period
        if maturity <= 0: return principal+principal*coupon_rate/coupons_per_year
        cash_flows = bond_cash_flows(maturity, principal, coupon_rate, coupons_per_year)
        return pv(cash_flows, discount_rate/coupons_per_year) 

Question 1
In the following questions, we will be working with three bonds:

B1 is a 15 Year Bond with a Face Value of $1000 that pays a 5% coupon semi-annually (2 times a year)
B2 is a 5 Year Bond with a Face value of $1000 that pays a 6% coupon quarterly (4 times a year)
B3 is a 10 Year Zero-Coupon Bond with a Face Value of $1000 (Hint: you can still use the erk.bond_cash_flows() and erk.bond_price() by setting the coupon amount to 0% and coupons_per_year to 1)  Assume the yield curve is flat at 5%. Duration refers to Macaulay Duration
Hint: the macaulay_duration function gives as output the duration expressed in periods and not in years. If you want to get the yearly duration you need to divide the duration for coupons_per_year; 

e.g.: duarion_B2 = erk.macaulay_duration(flows_B2, 0.05/4)/4

Which of the three bonds is the most expensive?

B3

[x] B2

B1
1 point

In [4]:
b1 = bond_price(15, 1000, .05, 2,.05 )
b2 = bond_price(5, 1000, .06, 4,.05 )
b3 = bond_price(10, 1000, .0, 1,.05 )
bond_prices = pd.DataFrame([b1, b2, b3],index = ["b1","b2","b3"])
bond_prices

,0
b1,1000.000000
b2,1043.998290
b3,613.913254


In [5]:
bond_prices.idxmax()

0    b2
dtype: object

Question 2
Which of the three bonds is the least expensive?

B2

B1

[x] B3
1 point

In [6]:
bond_prices.idxmin()

0    b3
dtype: object

3 What is the price of the 10 Year Zero Coupon Bond B3?

In [7]:
round(b3)

0    614.0
dtype: float64

Question 4
Which of the three bonds has the highest (Macaulay) Duration?

B3

B2

[x] B1
1 point

In [8]:
def macaulay_duration(flows, discount_rate):
    """
    Computes the Macaulay Duration of a sequence of cash flows, given a per-period discount rate
    """
    discounted_flows = discount(flows.index, discount_rate)*pd.DataFrame(flows)
    weights = discounted_flows/discounted_flows.sum()
    return np.average(flows.index, weights=weights.iloc[:,0])

In [9]:
md_b1 = macaulay_duration(bond_cash_flows(15, 1000, .05, 2),  .05/2) /2
md_b2 = macaulay_duration(bond_cash_flows(5, 1000, .06, 4 ),  .05/4) / 4
md_b3 = macaulay_duration(bond_cash_flows(10, 1000, .0, 1 ),  .05)
bond_dur = pd.DataFrame([md_b1, md_b2, md_b3],index = ["b1","b2","b3"])
bond_dur

,0
b1,10.726775
b2,4.373363
b3,10.000000


In [10]:
bond_dur.idxmax()

0    b1
dtype: object

Question 5
Which of the three bonds has the lowest (Macaulay) Duration?

[x] B2

B3

B1
1 point

In [11]:
bond_dur.idxmin()

0    b2
dtype: object

6 What is the duration of the 5 year bond B2?

In [12]:
md_b2.round(2)

4.37

7 Assume a sequence of 3 liabilities of $100,000, $200,000 and $300,000 that are 3, 5 and 10 years away,
 respectively. What is the Duration of the liabilities?

(Reminder:  Assume the yield curve is flat at 5%. Duration refers to Macaulay Duration)

In [13]:
liabilities = pd.Series(data = [100000, 200000, 300000], index=[3,5,10])
liab_dur = macaulay_duration(liabilities, .05)
liab_dur.round(2)

6.75

8 Assuming the same set of liabilities as the previous question (i.e. a sequence of 3 liabilities of $100,000, $200,000 and $300,000 that are 3, 5 and 10 years away,
 respectively) build a Duration Matched Portfolio of B1 and B2 to match these liabilities. What is the weight of B2 in the portfolio? (Hint: the code
 we developed in class erk.match_durations() assumes that all the bonds have the same number of coupons
 per year. This is not the case here, so you will either need to enhance the code or compute the weight directly
 e.g. by entering the steps in a Jupyter Notebook Cell or at the Python Command Line)

In [14]:
weight_short_term_bond = (md_b1 - liab_dur)/(md_b1 - md_b2)
(100*weight_short_term_bond).round(2)

62.58

Question 9
Assume you can use any of the bonds B1, B2 and B3 to build a duration matched bond portfolio matched
 to the liabilities. Which combination of 2 bonds can you NOT use to build a duration matched bond portfolio?

[] ANY PAIR WILL WORK

[] B2+B3

[x] B1+B3

B1+B2
1 point

In [17]:
# weight_short_term_bond  = (md_b3 - liab_dur)/(md_b3 - md_b1)
# (100*weight_short_term_bond).round(2)

10 Assuming the same liabilities as the previous questions (i.e. a sequence of 3 liabilities of $100,000, $200,000 and $300,000 that are 3, 5 and 10 years away,
 respectively), build a Duration Matched Portfolio of B2 and B3 to match the liabilities.

What is the weight of B2 in this portfolio?

In [16]:
weight_short_term_bond  = (md_b3 - liab_dur)/(md_b3 - md_b2)
(100*weight_short_term_bond).round(2)

57.74